# Processing Time Analysis
This notebook allows you to select and visualize processing time and read time data from marker location files.

In [57]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import os
from pathlib import Path

## Setup
Configure the base directory and define functions to find folders containing MarkerLocationsGA_CouchShift files.

In [58]:
# Base directory for data
base_dir = r"C:\Users\kankean.kandasamy\Repo\KIM-QA-Analysis\Lyrebird Data"

# Column names for MarkerLocationsGA files (even if headers are missing)
GA_COLUMNS = [
    'Frame No', 'Time (sec)', 'Gantry', 
    'Marker_0_AP', 'Marker_0_LR', 'Marker_0_SI',
    'Marker_1_AP', 'Marker_1_LR', 'Marker_1_SI',
    'Marker_2_AP', 'Marker_2_LR', 'Marker_2_SI',
    'Marker_0_X', 'Marker_0_Y', 'Marker_1_X', 'Marker_1_Y', 'Marker_2_X', 'Marker_2_Y',
    'Marker_0_Correlation', 'Marker_0_Segmentation',
    'Marker_1_Correlation', 'Marker_1_Segmentation',
    'Marker_2_Correlation', 'Marker_2_Segmentation',
    'Joint Template', 'Processing Time (ms)', 'Read Time (ms)',
    'HeaderTimeTag', 'Filename'
]

# Find all subdirectories
def find_folders(base_path):
    folders = []
    for root, dirs, filenames in os.walk(base_path):
        # Check if this directory contains MarkerLocationsGA_CouchShift files
        has_ga_files = any('MarkerLocationsGA_CouchShift' in f and f.endswith('.txt') for f in filenames)
        if has_ga_files:
            relative_path = os.path.relpath(root, base_path)
            folders.append((relative_path, root))
    return sorted(folders)

# Find all MarkerLocationsGA_CouchShift files in a folder
def find_ga_files_in_folder(folder_path):
    files = []
    for filename in os.listdir(folder_path):
        if 'MarkerLocationsGA_CouchShift' in filename and filename.endswith('.txt'):
            full_path = os.path.join(folder_path, filename)
            files.append((filename, full_path))
    return sorted(files)

available_folders = find_folders(base_dir)
print(f"Found {len(available_folders)} folders with MarkerLocationsGA_CouchShift files")

Found 17 folders with MarkerLocationsGA_CouchShift files


In [59]:
def plot_data(file_path, folder_name, x_axis_col):
    """Read data file and create interactive plots"""
    try:
        # Try reading with headers first
        try:
            df = pd.read_csv(file_path, skipinitialspace=True)
            df.columns = df.columns.str.strip()
            # Check if we have the expected columns
            if 'Processing Time (ms)' not in df.columns:
                raise ValueError("Missing expected columns")
        except:
            # Read without headers and assign column names
            df = pd.read_csv(file_path, header=None, names=GA_COLUMNS, skipinitialspace=True)
        
        # Extract relevant columns
        x_data = df[x_axis_col]
        processing_time = df['Processing Time (ms)']
        read_time = df['Read Time (ms)']
        
        # Create subplots with 2 rows
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Processing Time', 'Read Time'),
            vertical_spacing=0.12,
            shared_xaxes=True
        )
        
        # Add Processing Time trace (top panel)
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=processing_time,
                mode='lines+markers',
                name='Processing Time',
                line=dict(color='#1f77b4', width=2),
                marker=dict(size=4),
                hovertemplate=f'{x_axis_col}: %{{x}}<br>Processing Time: %{{y:.2f}} ms<extra></extra>'
            ),
            row=1, col=1
        )
        
        # Add Read Time trace (bottom panel)
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=read_time,
                mode='lines+markers',
                name='Read Time',
                line=dict(color='#ff7f0e', width=2),
                marker=dict(size=4),
                hovertemplate=f'{x_axis_col}: %{{x}}<br>Read Time: %{{y:.2f}} ms<extra></extra>'
            ),
            row=2, col=1
        )
        
        # Update axes labels
        fig.update_xaxes(title_text=x_axis_col, row=2, col=1)
        fig.update_yaxes(title_text='Processing Time (ms)', row=1, col=1)
        fig.update_yaxes(title_text='Read Time (ms)', row=2, col=1)
        
        # Update layout with foldername - filename
        filename = os.path.basename(file_path)
        fig.update_layout(
            height=800,
            showlegend=True,
            hovermode='x unified',
            title=dict(
                text=f'<b>{folder_name} - {filename}</b>',
                x=0.5,
                xanchor='center'
            ),
            font=dict(size=12)
        )
        
        # Enable pan and zoom
        fig.update_xaxes(rangeslider_visible=False)
        
        return fig, df
        
    except Exception as e:
        print(f"Error reading file {os.path.basename(file_path)}: {e}")
        return None, None

def on_plot_button_clicked(b):
    """Handle plot button click"""
    with output:
        output.clear_output(wait=True)
        selected_folder = folder_dropdown.value
        x_axis = xaxis_radio.value
        
        if selected_folder:
            folder_name = os.path.basename(selected_folder)
            ga_files = find_ga_files_in_folder(selected_folder)
            
            if not ga_files:
                print("No MarkerLocationsGA_CouchShift files found in the selected folder.")
                return
            
            print(f"Processing {len(ga_files)} file(s) from: {folder_name}\n")
            
            for filename, file_path in ga_files:
                print(f"Loading: {filename}")
                fig, df = plot_data(file_path, folder_name, x_axis)
                
                if fig is not None:
                    fig.show()
                    print(f"\nData Summary for {filename}:")
                    print(f"Total Frames: {len(df)}")
                    print(f"\nProcessing Time (ms):")
                    print(f"  Mean: {df['Processing Time (ms)'].mean():.2f}")
                    print(f"  Median: {df['Processing Time (ms)'].median():.2f}")
                    print(f"  Min: {df['Processing Time (ms)'].min():.2f}")
                    print(f"  Max: {df['Processing Time (ms)'].max():.2f}")
                    print(f"\nRead Time (ms):")
                    print(f"  Mean: {df['Read Time (ms)'].mean():.2f}")
                    print(f"  Median: {df['Read Time (ms)'].median():.2f}")
                    print(f"  Min: {df['Read Time (ms)'].min():.2f}")
                    print(f"  Max: {df['Read Time (ms)'].max():.2f}")
                    print("\n" + "="*80 + "\n")
        else:
            print("Please select a folder to plot.")

# Create dropdown widget for folder selection
folder_dropdown = widgets.Dropdown(
    options=[(name, path) for name, path in available_folders],
    description='Select Folder:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Create radio buttons for x-axis selection
xaxis_radio = widgets.RadioButtons(
    options=['Frame No', 'Time (sec)'],
    value='Time (sec)',
    description='X-Axis:',
    disabled=False
)

# Create button to trigger plot
plot_button = widgets.Button(
    description='Plot All Files',
    button_style='primary',
    tooltip='Click to plot all MarkerLocationsGA_CouchShift files in the selected folder',
    icon='chart-line'
)

output = widgets.Output()

plot_button.on_click(on_plot_button_clicked)

display(folder_dropdown)
display(xaxis_radio)
display(plot_button)
display(output)

Dropdown(description='Select Folder:', layout=Layout(width='80%'), options=(('2025-03-05 CTRO Static Tests Rep…

RadioButtons(description='X-Axis:', index=1, options=('Frame No', 'Time (sec)'), value='Time (sec)')

Button(button_style='primary', description='Plot All Files', icon='chart-line', style=ButtonStyle(), tooltip='…

Output()

## Plotting Functions
Define the data reading and plotting logic. This cell must be executed before clicking the "Plot All Files" button.

## Instructions
1. Select a folder from the dropdown menu
2. Choose the X-axis variable (Frame No or Time)
3. Click 'Plot All Files' to visualize all MarkerLocationsGA_CouchShift files in the folder
4. Each file will be plotted separately with its own statistics
5. Use Plotly's interactive features:
   - **Pan**: Click and drag
   - **Zoom**: Use scroll wheel or click and drag to select area
   - **Box/Lasso Select**: Click mode icons in toolbar
   - **Reset**: Double-click the plot
   - **Hover**: Move mouse over data points for details